In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import re
import requests
import os

In [2]:
navegador = webdriver.Chrome()

In [3]:
navegador.get('https://google.com.br')

In [4]:
barra_pesquisa = navegador.find_element(By.NAME, 'q')

In [5]:
barra_pesquisa.send_keys('Campeonato brasileiro 2025')

In [6]:
barra_pesquisa.send_keys(Keys.ENTER)

In [7]:
link = navegador.find_element(By.PARTIAL_LINK_TEXT, "Tabela e Jogos do Brasileirão Série A 2025")
link.click()

In [44]:
linhasEstatisticas = navegador.find_elements(By.CSS_SELECTOR, ".tabela__pontos tbody tr")
clubes = navegador.find_elements(By.CSS_SELECTOR, ".classificacao__equipes--nome")

In [28]:
goleadores = navegador.find_elements(By.CSS_SELECTOR, ".ranking-item-wrapper")
for i, goleador in enumerate(goleadores[:10], start=1):
    nome = goleador.find_element(By.CSS_SELECTOR, ".jogador-nome")
    gols = goleador.find_element(By.CSS_SELECTOR, ".jogador-gols")
    posicao = goleador.find_element(By.CSS_SELECTOR, ".jogador-posicao")
    imagem = goleador.find_element(By.CSS_SELECTOR, ".jogador-escudo img")
    time = imagem.get_attribute("alt")
    print(f"{i}. {nome.text} {posicao.text} com {gols.text} - {time}")



1. Arrascaeta MEIO-CAMPO com 9 - Flamengo
2. Kaio Jorge ATACANTE com 8 - Cruzeiro
3. Vegetti ATACANTE com 7 - Vasco
4. Pedro Raul ATACANTE com 6 - Ceará
5. Reinaldo LATERAL-ESQUERDO com 6 - Mirassol
6. Yuri Alberto ATACANTE com 5 - Corinthians
7. Isidro Pitta ATACANTE com 4 - Bragantino
8. Pedro ATACANTE com 4 - Flamengo
9. Rony ATACANTE com 3 - Atlético-MG
10. Lucero ATACANTE com 3 - Fortaleza


In [45]:
campos = [
    "pontos",
    "jogos",
    "vitórias",
    "empates",
    "derrotas",
    "gols_pro",
    "gols_contra",
    "saldo_de_gols",
    "aproveitamento"
]

In [46]:
tabela_classificacao = {}
cabecalho = f"{'Clube':<25} {'P':>3} {'J':>3} {'V':>3} {'E':>3} {'D':>3} {'GP':>4} {'GC':>4} {'SG':>4} {'AP':>5}"

if len(clubes) == len(linhasEstatisticas):
    for clube, estatistica in zip(clubes, linhasEstatisticas):
        valores = estatistica.text.split()
        dados_clube = {campo: valor for campo, valor in zip(campos, valores)}
        tabela_classificacao[clube.text] = dados_clube
else:
    print("Erro: número de clubes e estatísticas não corresponde.")

In [47]:
def mostrar_tabela():
    print(cabecalho)
    print("-" * len(cabecalho))

    for clube, estatisticas in tabela_classificacao.items():
        print(f"{clube:<25} "
            f"{estatisticas['pontos']:>3} "
            f"{estatisticas['jogos']:>3} "
            f"{estatisticas['vitórias']:>3} "
            f"{estatisticas['empates']:>3} "
            f"{estatisticas['derrotas']:>3} "
            f"{estatisticas['gols_pro']:>4} "
            f"{estatisticas['gols_contra']:>4} "
            f"{estatisticas['saldo_de_gols']:>4} "
            f"{estatisticas['aproveitamento']:>4}%")

In [48]:
def estatisticas_time():
    print("Times disponíveis:\n")
    for i, clube in enumerate(tabela_classificacao.keys(), start=1):
        print(f"{i}. {clube}")

    escolha = int(input("\nDigite o número do time desejado: ")) - 1

    nomes_times = list(tabela_classificacao.keys())
    if 0 <= escolha < len(nomes_times):
        time = nomes_times[escolha]
        dados = tabela_classificacao[time]

        texto = (
            f"\n{time} disputou {dados['jogos']} jogos.\n"
            f"Conquistou {dados['vitórias']} vitórias, "
            f"{dados['empates']} empates e {dados['derrotas']} derrotas.\n"
            f"Marcou {dados['gols_pro']} gols e sofreu {dados['gols_contra']}.\n"
            f"Tem um saldo de gols de {dados['saldo_de_gols']}.\n"
            f"Está na {escolha+1} posição com {dados['pontos']} pontos e um aproveitamento de {dados['aproveitamento']}%."
        )
        print(texto)
    else:
        print("Número inválido. Por favor, escolha um número da lista.")


In [49]:
jogos = navegador.find_elements(By.CSS_SELECTOR, ".lista-jogos__jogo")


In [50]:
jogos_disponiveis = []

def list_jogos():
    for idx, jogo in enumerate(jogos):
        linhas = jogo.text.split('\n')
        linhas = [linha for linha in linhas if linha.strip() and "FIQUE POR DENTRO" not in linha]

        if len(linhas) >= 3:
            linha0 = linhas[0]
            padrao = re.compile(r"(.*?)(\d{2}/\d{2})([A-Za-zÀ-ÿ]+)?(\d{2}:\d{2})")
            m = padrao.match(linha0)

            if m:
                local = m.group(1).strip()
                data = m.group(2)
                dia = m.group(3) if m.group(3) else ""
                hora = m.group(4)

                if len(linhas) == 3:
                    # Jogo futuro (sem placar)
                    time_casa = linhas[1]
                    time_visitante = linhas[2]
                    placar = None
                elif len(linhas) >= 6:
                    # Jogo encerrado
                    time_casa = linhas[1]
                    gols_casa = linhas[2]
                    gols_visitante = linhas[3]
                    time_visitante = linhas[4]
                    placar = f"{gols_casa} x {gols_visitante}"
                else:
                    print(f"{idx + 1}. Formato inesperado (linhas insuficientes):", linhas)
                    continue

                jogos_disponiveis.append({
                    "time_casa": time_casa,
                    "time_visitante": time_visitante,
                    "local": local,
                    "data": data,
                    "dia": dia,
                    "hora": hora,
                    "placar": placar
                })

                texto_extra = f" - Placar: {placar}" if placar else ""
                print(f"{idx + 1}. {time_casa} x {time_visitante} - {data} {hora} ({local}){texto_extra}")
            else:
                print(f"{idx + 1}. Erro ao ler local/data/hora: {linha0}")
        else:
            print(f"{idx + 1}. Formato inesperado:", linhas)

    # while True:
    #     try:
    #         escolha = int(input("\nDigite o número do jogo desejado: "))
    #         if 1 <= escolha <= len(jogos_disponiveis):
    #             break
    #         else:
    #             print("Número fora do intervalo.")
    #     except ValueError:
    #         print("Entrada inválida. Digite um número.")

    # jogo_selecionado = jogos_disponiveis[escolha - 1]

    # dadosCasa = tabela_classificacao[jogo_selecionado['time_casa']]
    # dadosVisita = tabela_classificacao[jogo_selecionado['time_visitante']]

    # # Prompt para API ou exibição
    # prompt = (
    #     f"Considere o jogo entre {jogo_selecionado['time_casa']} e {jogo_selecionado['time_visitante']}.\n"
    #     f"Local: {jogo_selecionado['local']}\n"
    #     f"Data: {jogo_selecionado['data']} {jogo_selecionado['dia']}\n"
    #     f"Hora: {jogo_selecionado['hora']}.\n\n"
    #     f"\nDesempenho Atual do {jogo_selecionado['time_casa']}:\n"
    #     f"- Jogos disputados: {dadosCasa['jogos']}\n"
    #     f"- Vitórias: {dadosCasa['vitórias']}, Empates: {dadosCasa['empates']}, Derrotas: {dadosCasa['derrotas']}\n"
    #     f"- Gols marcados: {dadosCasa['gols_pro']}, Gols sofridos: {dadosCasa['gols_contra']}\n"
    #     f"- Saldo de gols: {dadosCasa['saldo_de_gols']}\n"
    #     f"- Pontuação: {dadosCasa['pontos']} pontos\n"
    #     f"- Aproveitamento: {dadosCasa['aproveitamento']}%\n"
    #     f"\nDesempenho Atual do {jogo_selecionado['time_visitante']}:\n"
    #     f"- Jogos disputados: {dadosVisita['jogos']}\n"
    #     f"- Vitórias: {dadosVisita['vitórias']}, Empates: {dadosCasa['empates']}, Derrotas: {dadosCasa['derrotas']}\n"
    #     f"- Gols marcados: {dadosVisita['gols_pro']}, Gols sofridos: {dadosCasa['gols_contra']}\n"
    #     f"- Saldo de gols: {dadosVisita['saldo_de_gols']}\n"
    #     f"- Pontuação: {dadosVisita['pontos']} pontos\n"
    #     f"- Aproveitamento: {dadosVisita['aproveitamento']}%\n"
    #     "Baseado no desempenho recente dos dois times e o local do jogo, faça uma previsão realista do resultado da partida."
    # )
    # print(prompt)


In [ ]:

# API_KEY = "sUA CHAVE"

# # Endpoint da API ChatGPT (GPT-4)
# url = "https://api.openai.com/v1/chat/completions"

# # Função para chamar a API e obter uma previsão
# def obter_previsao_jogo():

#     headers = {
#         "Authorization": f"Bearer {API_KEY}",
#         "Content-Type": "application/json"
#     }

#     body = {
#         "model": "gpt-3.5-turbo",
#         "messages": [
#             {"role": "system", "content": "Você é um especialista em futebol brasileiro."},
#             {"role": "user", "content": prompt}
#         ],
#         "temperature": 0.7
#     }

#     response = requests.post(url, headers=headers, json=body)

#     if response.status_code == 200:
#         resposta = response.json()
#         return resposta["choices"][0]["message"]["content"]
#     else:
#         return f"Erro {response.status_code}: {response.text}"
# previsao = obter_previsao_jogo()
# print("\n🔮 Previsão da IA para o jogo:\n")
# print(previsao)


🔮 Previsão da IA para o jogo:

Erro 429: {
    "error": {
        "message": "You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.",
        "type": "insufficient_quota",
        "param": null,
        "code": "insufficient_quota"
    }
}



In [ ]:
def exibir_menu():
    print("\n======= MENU PRINCIPAL =======")
    print("1 - Mostrar tabela")
    print("2 - Estatitisticas do time")
    print("3 - Jogos da Rodada")
    print("5 - Sair")

def main():
    while True:
        exibir_menu()
        opcao = input("Escolha uma opção: ")

        if opcao == '1':
            mostrar_tabela()
        elif opcao == '2':
            estatisticas_time()
        elif opcao == '3':
            list_jogos()
        elif opcao == '4':
            prever()
        elif opcao == '5':
            print("Saindo do programa...")
            break
        else:
            print("Opção inválida. Tente novamente.")

if __name__ == "__main__":
    main()


======= MENU PRINCIPAL =======
1 - Mostrar tabela
2 - Estatitisticas do time
3 - Jogos da Rodada
5 - Sair
1. FLU x VAS - 24/05 18:30 (Maracanã) - Placar: 2 x 1
2. SAO x MIR - 24/05 18:30 (Morumbis) - Placar: 0 x 2
3. CAM x COR - 24/05 21:00 (Arena MRV) - Placar: 0 x 0
4. GRE x BAH - 25/05 11:00 (Arena do Grêmio) - Placar: 1 x 0
5. PAL x FLA - 25/05 16:00 (Allianz Parque) - Placar: 0 x 2
6. SPT x INT - 25/05 16:00 (Ilha do Retiro) - Placar: 1 x 1
7. VIT x SAN - 25/05 18:30 (Barradão) - Placar: 0 x 1
8. FOR x CRU - 25/05 20:30 (Castelão (CE)) - Placar: 0 x 2
9. RBB x JUV - 26/05 20:00 (Cícero de Souza Marques) - Placar: 1 x 0
10. BOT x CEA - 04/06 20:00 (Nilton Santos (Engenhão))

======= MENU PRINCIPAL =======
1 - Mostrar tabela
2 - Estatitisticas do time
3 - Jogos da Rodada
5 - Sair
Saindo do programa...
